In [ ]:
##Longhand - M. Cook - 2022
##Takes text corpus and returns immersive visualization
##https://github.com/Cook4986/Longhand

In [ ]:
#Generates bag-of-words from individual image transcriptions
import os
import sys
import re
from pathlib import Path

#input/output
paths = '....txt'#plain text list of urls or filenames (pre-existing)
target = '...' #HP outputs (pre-existing)
textOut = '....txt'#Bag-of-Words output

#declarations
BoW = open(textOut, "a")

#append bag-of-words with headers and transcriptions
for path in sorted(Path(target).rglob('*.txt')):
    header = path.stem.split('.')
    BoW.write("\n")
    BoW.write(header[0]) #write headers location to bag-of-words
    BoW.write("\n")
    print((header[0]) + " added to bag-of-words")
    absolute = (str(path.parent) + "/" + path.name)
    contents = open(absolute, "r") 
    BoW.write(str(contents.read())) #write transcriptions to bag-of-words
    print(("Transcription for " + header[0]) + " added to bag-of-words")
    BoW.write("\n")
BoW.close()
print("Bag-of-words generated and output from corpus")


In [ ]:
#Takes corpus Bag-of-Words and returns json dump of common nouns, Sketchfab models/uids, and relative percentage of occurance
import spacy
from collections import Counter
import json
import requests
from requests.exceptions import RequestException
import time

#declarations
nlp = spacy.load("en_core_web_sm")
BoW = "...txt"
SKETCHFAB_API_URL = "https://api.sketchfab.com/v3/search?type=models&count=1&max_face_count=1000" #note count parameter
API_TOKEN = '...'
output = "...txt"
results = 5
start = time.time()

#data structures
nouns = [] # all nounds in Bag-of-Words
freqs = [] # noun appearances in BoW
objects = {} # key = [common nouns]; value(s) = [uids, percentage, model name]
    
#parse Bag-of-Words parts-of-speech with Spacy (english)
with open(BoW, encoding="utf-8") as file:
    iliad = file.read()
document = nlp(iliad)
document.text
for token in document:
    if token.pos_ == 'NOUN':
        nouns.append(token.text)
file.close()

#frequency & dictionary with common words and percentages
word_freq = Counter(nouns)
common = word_freq.most_common(results)
for word in common:
    freqs.append(word[1])
Sum = sum(freqs)
for word in common:
    flowt = (word[1] / Sum) * 100
    percentage = round(flowt, 2)
    objects[word[0]] = [percentage]

#Sketchfab API payload function 
##From https://sketchfab.com/developers/data-api/v3/python#example-python-model
def _get_request_payload(*, data=None, files=None, json_payload=False):
    """Helper method that returns the authentication token and proper content type depending on
    whether or not we use JSON payload."""
    data = data or {}
    files = files or {}
    headers = {'Authorization': 'Token {}'.format(API_TOKEN)}
    if json_payload:
        headers.update({'Content-Type': 'application/json'})
        data = json.dumps(data)
    return {'data': data, 'files': files, 'headers': headers}

#query sketchfabs with nouns in BoW and return/write list of uids + model names
for key in objects.keys():
    print("Searching: " + str(key))
    print("\n")
    query = ("&q="+(str(key))+"&downloadable=true")
    search_endpoint = f'{SKETCHFAB_API_URL + query}'
    payload = _get_request_payload() 
    response = requests.get(search_endpoint, **payload)
    data = response.json()
    for item in range(len(data['results'])):
        uid = (data['results'][item]['uid'])
        name = (str((data['results'][item]['name'])))
        objects[key] += [name, uid]
        print("the following model been located: ")
        print(str((data['results'][item]['name']))+(" \nuid: ")+(data['results'][item]['uid']))
        print("\n")

#write to disk and close program
with open(output, 'w') as file:
    file.write(json.dumps(objects)) 
file.close()
print(str(results) + " objects written to disk")
print("\n")
end = time.time()
print(str(end - start) + " seconds elapsed" )
print("\n")
print("have a nice day")


In [ ]:
##Launches Blender from terminal and initiates model download script
!.../Blender --python .../Longhand_downloader.py


In [ ]:
##TO DO:
#Github:
    ##Combine with SAEF (for HTR)
    ##Begin Longhand with BoW
#Blender:
    ##Add ground plane
    ##Distribute sketchfab models
    ##Append Hubs components
#Python/AWS
    ##generate URL of hosted GLB

#Research: 
##viewing angle limitations and text legibility)
##                     vs.
##non-accidental (3D) object properties and object recognition
